# Use Case 1: Defibrillator data from real-world out-of-hospital cardiac arrest cases
#### This notebook illustrates the usage of the pyvital class to visualize, annotate and process time-series data from the medical field.
#### In this case we analyze data recorded by a defibrillator during a real-world out-of-hospital cardiac arrest case.


In [4]:
import bz2
import tempfile
import shutil

from vitabel import Vitals, Label
from pathlib import Path

## 1) Load Data

An empty pyvital object is initiated and all the channels from a defibrillator recording (in this case a ZOLL X-Series case, stored in a `bz2`-compressed file) are loaded.

In [ ]:
case = Vitals()

compressed_defi_file = Path("data/ZOLL_test_case.json.bz2")
with tempfile.TemporaryDirectory() as tmpdir:
    defi_file = Path(tmpdir) / compressed_defi_file.stem
    with bz2.open(compressed_defi_file, "rb") as source:
        with open(defi_file, "wb") as dest:
            shutil.copyfileobj(source, dest)

    case.add_defibrillator_recording(defi_file)

 We get an overview over all loaded channels by calling case.info()

In [ ]:
case.metadata

In [ ]:
case.info()

## 2) Process Data

 We use the integrated methods to determine etco2 values from the capnography signal. \
Furthermore we use the cpr_acceleration signal and the ecg from the defibrillation pads to determine predictions for the circulatory state

In [ ]:
case.compute_etco2_and_ventilations()
case.predict_circulation()

We want to annotate occurances of Return of spontaneous circulation (ROSC), thus we create a label and add it to our case. 


In [ ]:
ROSC_label = Label(
    "ROSC", plotstyle={"marker": "$\u2665$", "color": "red", "ms": 10, "linestyle": ""}
)
case.add_global_label(ROSC_label)

 We see the new labels if check out all labels present in our case

In [ ]:
case.get_label_names()

## 3) Interactively Plot Data


We provide a setup for the interactive plot. Then we check the validity of the etco2-values and additionally label the ROSCS which we think occured in this case. 

In [ ]:
plot = case.plot_interactive(
    channels=[["cpr_acceleration"], ["capnography"], ["ecg_pads"], []],
    labels=[
        ["ROSC"],
        ["etco2_from_capnography", "ROSC"],
        ["ROSC"],
        ["ROSC", "rosc_probability"],
    ],
    channel_overviews=[["cpr_acceleration"]],
    time_unit="s",
    subplots_kwargs={"figsize": (22, 9)},
)
plot

Show annotated ROSC data

In [ ]:
ROSC_label.get_data()

## 4) Store Data


In [ ]:
case.save_data("case_1.json")
case.get_label("ROSC").to_csv()